In [1]:
# Import quandl to retrieve data
# Pandas and matplotlib standard imports for data analysis
import quandl
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Use Plotly for better graphs
import plotly.offline as plotly
import plotly.graph_objs as go
import plotly.graph_objects as go

# Offline Plotting
plotly.init_notebook_mode(connected=True)

# Put your api key here
quandl.ApiConfig.api_key = 'g3sVyGQgGrx_J1RV9E43'
mapbox_access_token = 'pk.eyJ1IjoiY2hyaXN0aW5hdGhpcCIsImEiOiJjazB4ZGN2N3YwNTJuM21sYXd3eHZ1dmJiIn0.ogLmQDqwezijg16HL_9RTQ'

In [2]:
# Specify Zillow Indicator we want to graph
indicator = 'MLPAH'

# Specify the county and state we want to generate the map for
metro = 'Nashville'
state = 'TN'

# Start date and end date should be the same
start_date = '2010-09-01'
end_date = '2019-08-31'

In [3]:
# Get data for each zip code in the city and add that to a single dataframe
def get_city_data(cities, indicator):
    df_array = []
    for city, code in cities.items():
        quandl_db_code = f'ZILLOW/Z{code}_{indicator}'
        data = quandl.get(quandl_db_code, start_date=start_date, end_date=end_date)
        data.rename(index=str, columns={'Value': city}, inplace=True)
        data.index = pd.to_datetime(data.index, format='%Y/%m/%d')
        df_array.append(data)

    ret = pd.concat(df_array, axis=1)
    return ret

In [4]:
# use the logitude / latitude data for plotting

ZIP_CODES = pd.read_csv('nash_zipcode.csv') 
ZIP_CODES.head()

,zip_code,latitude,longitude,city,state
0,37013,36.03765,-86.63697,Nashville,TN
1,37027,36.00159,-86.78068,Nashville,TN
2,37072,36.35495,-86.76214,Nashville,TN
3,37076,36.14312,-86.58965,Nashville,TN
4,37115,36.25536,-86.69827,Nashville,TN


In [5]:
real_estate_data = get_city_data(ZIP_CODES['zip_code'], indicator)
print(real_estate_data)
real_estate_data.to_csv(r'nashville.csv')


                  0         1         2         3         4         5   \
Date                                                                     
2010-09-30  135900.0  474990.0       NaN  153400.0  109500.0       NaN   
2010-10-31  135000.0  475000.0       NaN  157900.0  109000.0       NaN   
2010-11-30  135200.0  479000.0       NaN  155000.0  104950.0       NaN   
2010-12-31  134894.0  484950.0       NaN  154900.0   99900.0       NaN   
2011-01-31  130750.0  489900.0       NaN  153000.0  108500.0       NaN   
2011-02-28  131995.0  462445.0       NaN  149900.0   99900.0       NaN   
2011-03-31  129999.0  479000.0       NaN  149900.0   99900.0       NaN   
2011-04-30  129900.0  479500.0       NaN  149900.0   99900.0       NaN   
2011-05-31  129900.0  489900.0       NaN  149900.0  102272.5       NaN   
2011-06-30  129900.0  479900.0       NaN  152990.0   99700.0       NaN   
2011-07-31  129900.0  489900.0       NaN  151441.0  105000.0       NaN   
2011-08-31  129875.0  489900.0       N

In [6]:
transformed_real_estate_data = real_estate_data.transpose()

column_name = f'{indicator} {end_date}'

In [7]:
transformed_real_estate_data[column_name] = transformed_real_estate_data[end_date].astype('float')

In [8]:
concat_data = pd.concat([ZIP_CODES, transformed_real_estate_data[column_name]], axis=1, sort=True)
concat_data.to_csv(r'nashville.csv')


In [10]:
# plotting using the Plotly Mapbox graph
df = concat_data

plot_data = [
    go.Scattermapbox(
        lon = df['longitude'],
        lat = df['latitude'],
        mode='markers',
        marker=dict(
            reversescale = True,
            autocolorscale = False,
            size=17,
            opacity=0.85,
            colorscale = "rdylbu",
            cmin = df[column_name].min(),
            color = df[column_name],
            cmax = df[column_name].max(),
            colorbar=dict(
                title="Price"
            )
        ),
        #text = df['text'],
        #hoverinfo= 'text'
    )]

layout = go.Layout(
    title= f'Median Listing Price for All Homes in {metro}, {state} {end_date}',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=df.iloc[0]['latitude'],
            lon=df.iloc[0]['longitude']
        ),
        pitch=0,
        zoom=8,
        style='dark'
    ),
)


In [11]:
fig = dict( data=plot_data, layout=layout )
plotly.iplot( fig, filename=f'Median Listing Price for All Homes in {metro} {state} {end_date}')
